In [2]:
import codecs
import numpy as np
import random
from sklearn import metrics as ms
from perceptron import Perceptron

Import successfully.


In [3]:
def read_data(s):
    f = codecs.open(s + '.features', 'r', 'utf-8')
    feas = [[s.strip().split('\t') for s in t.strip().split('\n')] for t in f.read().strip().split('\n\n')]
    for i in range(len(feas)):
        if not i % 1000:
            print(i)
        for j in range(len(feas[i])):
            for k in range(len(feas[i][j])):
                feas[i][j][k] = eval(feas[i][j][k])
    f.close()
    f = codecs.open(s + '.pos', 'r', 'utf-8')
    pos = [eval(t) for t in f.read().strip().split('\n')]
    f.close()
    labs = None
    if not s == 'test':
        f = codecs.open(s + '.labels', 'r', 'utf-8')
        labs = [[eval(r) for r in t.strip().split('\n')] for t in f.read().strip().split('\n\n')]
        f.close()
    return feas, pos, labs

def read_aux(s):
    f = codecs.open(s + '.props', 'r', 'utf-8')
    preds = [[s.strip().split('\t')[0] for s in t.strip().split('\n')] for t in f.read().strip().split('\n\n')]
    f.close()
    f = codecs.open(s + '.sampleIds', 'r', 'utf-8')
    sampleIds = eval(f.read())
    f.close()
    return preds, sampleIds

In [4]:
def labs2conll(bioes, wordList, sampleIds, outFile):
    p = 0
    f = codecs.open(outFile, 'w', 'utf-8')
    for i, words in enumerate(wordList):
        while sampleIds[p] < i:
            p += 1
        sampleOut = [[w] for w in words]
        currCol = 0
        while p < len(bioes) and sampleIds[p] == i:
            currCol += 1
            prevStat = 'O'
            prevLab = 'O'
            nLeft = 0
            nRight = 0
            for j, comb in enumerate(bioes[p]):
                
                if comb == 'O':
                    sampleOut[j] += ['*']
                    if prevStat == 'B':
                        sampleOut[j - 1][currCol] = '(%s*%s)' % (prevLab, prevLab)
                    elif prevStat == 'I':
                        sampleOut[j - 1][currCol] = '*)'
                    prevStat = 'O'
                    continue
                
                stat = re.findall("(.)-.*", comb)[0]
                lab = re.findall(".-(.*)", comb)[0]
                
                if j == len(bioes[p]) - 1:
                    if stat == 'B': 
                        stat = 'S'
                    elif stat == 'I':
                        stat = 'E'
                
                if stat in ('I', 'E'):
                    if prevStat == 'S':
                        sampleOut[j - 1][currCol] = '(%s*' % (prevLab)
                    elif prevStat in ('O', 'E'):
                        stat = 'B'
                
                if stat in ('B', 'S'):
                    if prevStat == 'B': 
                        sampleOut[j - 1][currCol] = '(%s*%s)' % (prevLab, prevLab)
                    elif prevStat == 'I':
                        sampleOut[j - 1][currCol] = '*)'
                
                if stat == 'I':
                    sampleOut[j] += ['*']
                elif stat == 'B':
                    sampleOut[j] += ['(%s*' % lab]
                    nLeft += 1
                elif stat == 'E':
                    sampleOut[j] += ['*)']
                    nRight += 1
                else:
                    sampleOut[j] += ['(%s*%s)' % (lab, lab)]
                
                prevStat = stat
                prevLab = lab
            if nLeft != nRight:  # Exceptions in prediction
                print(p)
            p += 1
        stringOut = '\n'.join(['\t'.join(t) for t in sampleOut])
        if stringOut.count('(') != stringOut.count(')'):
            print(stringOut)  # Check out exceptions
        f.write(stringOut)
        f.write('\n\n')
    f.close()

In [5]:
f = codecs.open('labelID_rev.dict')
label_dict = eval(f.read())
f.close()
model = Perceptron(434878, 26, 26, labelDict=label_dict)

In [148]:
trn_feas, trn_pos, trn_labs = read_data('trn')
model.train(trn_feas, trn_labs, trn_pos, 50, shuffle=True)
model.save('50epochs')

Iter    1: Accuracy=0.9845 -  1089346/ 1106544
Iter    2: Accuracy=0.9860 -  1091003/ 1106544
Iter    3: Accuracy=0.9873 -  1092467/ 1106544
Iter    4: Accuracy=0.9916 -  1097226/ 1106544
Iter    5: Accuracy=0.9912 -  1096779/ 1106544
Iter    6: Accuracy=0.9925 -  1098235/ 1106544
Iter    7: Accuracy=0.9935 -  1099364/ 1106544
Iter    8: Accuracy=0.9920 -  1097693/ 1106544
Iter    9: Accuracy=0.9919 -  1097636/ 1106544
Iter   10: Accuracy=0.9921 -  1097766/ 1106544
Iter   11: Accuracy=0.9941 -  1100041/ 1106544
Iter   12: Accuracy=0.9927 -  1098481/ 1106544
Iter   13: Accuracy=0.9949 -  1100894/ 1106544
Iter   14: Accuracy=0.9942 -  1100151/ 1106544
Iter   15: Accuracy=0.9926 -  1098334/ 1106544
Iter   16: Accuracy=0.9948 -  1100750/ 1106544
Iter   17: Accuracy=0.9948 -  1100740/ 1106544
Iter   18: Accuracy=0.9961 -  1102267/ 1106544
Iter   19: Accuracy=0.9967 -  1102941/ 1106544
Iter   20: Accuracy=0.9980 -  1104352/ 1106544
Iter   21: Accuracy=0.9983 -  1104715/ 1106544
Iter   22: Ac

KeyboardInterrupt: 

In [ ]:
dev_feas, dev_pos, dev_labs = read_data('dev')
dev_props, dev_sids =read_aux('dev')
model.eval(dev_feas, dev_labs, dev_pos, confMatFile='dev.confuse')
dev_predLabs = model.predict(dev_feas, dev_pos, names=True)
labs2conll(dev_predLabs, dev_props, dev_sids, 'dev.pred')

In [6]:
test_feas, test_pos, test_labs = read_data('test')
test_props, test_sids =read_aux('test')
test_predLabs = model.predict(test_feas, test_pos, names=True)
labs2conll(test_predLabs, test_props, test_sids, 'test.pred')

0
1000


FileNotFoundError: [Errno 2] No such file or directory: 'test.sampleIds'